<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/UnitCommitment(single_period).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit Commitment (single period)

This Jupyter Notebook solves an instance of the Unit Commitment Problem for a single period described in XXX

## Requirements

In [1]:
# PYOMO
!pip install pyomo 
import pyomo.environ as pe
# NEOS
import os
os.environ['NEOS_EMAIL'] = 'xxx@gmail.com'
# OTHER
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 3.4 MB/s eta 0:00:00


## Input Data

In [2]:
gen = pd.DataFrame({
       'unit': [1,   2], 
       'cost': [10,  20], 
       'pmin': [60,  20], 
       'pmax': [100, 100]})
dem = 50
ngen = len(gen)

## Solving the Unit Commitment

In [3]:
# Model
m = pe.ConcreteModel()
# Sets
m.g = pe.Set(initialize=list(range(ngen)))
# Variables
m.p = pe.Var(m.g,within=pe.NonNegativeReals)
m.u = pe.Var(m.g,within=pe.Binary)
def obj_rule(m):
  return sum(gen.loc[g,'cost']*m.p[g] for g in m.g)
m.obj = pe.Objective(rule=obj_rule)
# Energy balance
def bal_rule(m):
  return sum(m.p[g] for g in m.g) == dem
m.bal = pe.Constraint(rule=bal_rule)
# Minimum generation
def min_gen_rule(m,g):
  return m.p[g] >= m.u[g]*gen.loc[g,'pmin']
m.min_gen = pe.Constraint(m.g, rule=min_gen_rule)
# Maximum generation
def max_gen_rule(m,g):
  return m.p[g] <= m.u[g]*gen.loc[g,'pmax']
m.max_gen = pe.Constraint(m.g, rule=max_gen_rule)
# We solve the optimization problem using GLPK
pe.SolverManagerFactory('neos').solve(m,opt='bonmin').write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 4
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


## Solution Output

In [4]:
# Print results
print('obj =',m.obj())
print('p1 =',m.p[0].value)
print('p2 =',m.p[1].value)

obj = 999.9999999044458
p1 = 9.555419965740217e-09
p2 = 49.99999999044458
